In [1]:
import numpy as np

In [2]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [17]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)> 0:
            return False
        else:
            
            return True
    
    def add_queue(self, group):
        if group.get_vip():   # If current group is a VIP group, move it forward by four groups,
            enterQueue=False
            if len(self.groups) > 4:
                for i in range(0,4):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        enterQueue=True
                        break
                if (enterQueue is False):
                        self.groups.insert(4,group)
            elif len(self.groups) > 1 and len(self.groups) <= 4:
                for i in range(0,len(self.groups)):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        enterQueue=True
                        break
                if (enterQueue is False):
                    self.groups.insert(len(self.groups)-1,group)
            else: 
                self.groups.insert(0,group)
        else: 
            self.groups.insert(0,group) # If current group is NON-vip, 
            # if currently there is no group or only one group waiting in queue
            
            
    
    def del_queue(self): #delete last=delete first come group 
        return self.groups.pop()  

In [4]:
# this class can be regarded as "class printer" in that case.
class Table:
    def __init__(self, num, size):
        self.num= num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        
        self.currentGroup = None # Is the table occupied or not.
        
    def get_num(self):
        return self.num
    
    def busy(self):
        if self.currentGroup != None:
            return True
        else:
            return False      
    
    def startNext(self, newGroup):
        self.currentGroup = newGroup

In [5]:
class Group:
    def __init__(self, time, size, vip, groupID):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table= None # Which table the group will be assigned to
        self.timeRequest = mod_pert_random(0, 45, 120, samples=1).astype(int)  #How long will the group spend
        self.groupID=groupID
    
    def get_groupID(self):
        return self.groupID
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip
    
    def get_time_request(self): 
        return self.timeRequest

In [6]:
def tablesSetting(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Table(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Table(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Table(i+number_tables_2+number_tables_4,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [8]:
def generation(Duration):
    # Generate size 
    size = np.random.randint(1,7,30)
    print(size)
    
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    
    print(timestamp_list)
    
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    
    table_2,table_4,table_6=tablesSetting(6,4,2)
    
    total_timeR = [0]
    nextGroup_endTime_2={}
    
    # Add every group into corresponding queue
    groupNumb_2=0
    groupNumb_4=0
    groupNumb_6=0
    
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter], groupNumb_2))
                counter += 1
                groupNumb_2+=1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter], groupNumb_4))
                counter += 1
                groupNumb_4+=1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter], groupNumb_6))
                counter += 1
                groupNumb_6 += 1
                
                
        
    for i in range(Duration):   
        if i in nextGroup_endTime_2.values():
            for num in nextGroup_endTime_2.keys():
                if i == nextGroup_endTime_2[num]:
                    print('Clean:', num.num,'\n')
                    table_2[num.num].currentGroup = None
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
            
        
        if (len(queue_2.groups)>0):
            current_q=[]
            for item in queue_2.groups:
                current_q.append(item.get_groupID())
            print(current_q)
            if (i >= (queue_2.groups[len(queue_2.groups)-1].get_stamp())):
                for t in table_2:
                    if (len(queue_2.groups)>0) and (not t.busy()) and (i>=(queue_2.groups[len(queue_2.groups)-1].get_stamp())):
                        print('Head GroupID:',queue_2.groups[len(queue_2.groups)-1].get_groupID())
                        nextGroup = queue_2.del_queue()
                     
                        print('Wait time:', nextGroup.wait_time(i))
                     
                     
                     
                        t.startNext(nextGroup)
                        print('Group No.:', nextGroup.get_groupID())
                        print('VIP?:', nextGroup.get_vip())
                        print('Registration time:', nextGroup.get_stamp())
                        print('Entering into table time:',i)
                         
                         
                         
                        nextGroup.table=t.num
                        print('Table No.', t.num)
                         
                        nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
                        print('\n')
                        total_timeR.append(nextGroup.timeRequest+2)
        
    avg_timeR = sum(total_timeR)/len(total_timeR)
        
    return (queue_2, queue_4, queue_6)

In [9]:
output=generation(240)

[5 3 6 5 6 2 4 1 3 3 3 3 3 5 5 5 5 6 4 5 3 6 5 4 3 5 1 2 5 5]
[ 58 150 178  39 133 160  64  90 171 172  66  77  21 137  80 106 211  68
 141 144  60 172 101 224  74 159 118  79 134 135]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1, 0]
[3, 2, 1,

In [10]:
output[0].groups

[]

In [14]:
def generation(Duration):
    size = np.random.randint(1,3,30)  #1-6 people for one group, and generate x groups
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    table_2,table_4,table_6=tablesSetting(6,4,2)  #setting tables
    
    # print(table_2,table_4,table_6)
    total_timeR_2 = []
    nextGroup_endTime_2 ={}
    total_timeR_4 = []
    nextGroup_endTime_4 ={}
    total_timeR_6 = []
    nextGroup_endTime_6 ={}
    
    groupNumb_2=1
    groupNumb_4=1
    groupNumb_6=1
    
    for i in range(Duration):
        #print(i,'/n')
        if i in timestamp_list:
            #print(size[counter],'**')
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter],groupNumb_2))
                
                print('New Regislation vip', vip[counter])
                print('No.', groupNumb_2)
                print('Time', i)
                
                counter += 1
                groupNumb_2 += 1
                
                current_q=[]
                for item in queue_2.groups:
                    current_q.append(item.get_groupID())
                print(current_q, 'current queue_2')
                
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter],groupNumb_4))
                counter += 1
                groupNumb_4 += 1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter],groupNumb_6))
                counter += 1
                groupNumb_6 += 1
        
        #mark 4,6 then 2 can run        
        stimulation(i, table_2, queue_2, Duration, total_timeR_2, nextGroup_endTime_2)
        #stimulation(i, table_4, queue_4, Duration, total_timeR_4, nextGroup_endTime_4)
        #stimulation(i, table_6, queue_6, Duration, total_timeR_6, nextGroup_endTime_6)

        
        # if i in nextGroup_endTime_2.values():
        #     for n in list(nextGroup_endTime_2.keys()):
        #         if i == int(nextGroup_endTime_2[n]):
        #             print('Clean:', n,'\n')
        #             table_2[n-1].currentGroup = None
        #     #table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
        #     #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
                    
        
        # for t in table_2:
        #     if (not t.busy()) and (not queue_2.isEmpty()):
        #         print(t.get_num(),'table ID')
        #         nextGroup = queue_2.del_queue()
        #         print('Regislation time:', nextGroup.get_stamp())
        #         print('Wait time:', nextGroup.wait_time(i))
        #         t.startNext(nextGroup)
        #         print('Group can sit now:', nextGroup.get_groupID())
        #         nextGroup_endTime_2[t.get_num()] = i+nextGroup.get_time_request()+2  #end time for each 2-people-table
        #         #2min for table cleaning, rearragement and the time to walk in blabla
        #         #print(nextGroup_endTime_2[t.get_num()], 'next group end time')
        #         total_timeR.append(int(nextGroup.get_time_request())+2)  #def generation(Duration):
        #         #print(total_timeR,'total request time \n')

    # at_least_waittime = []
    # for i in range(queue_2.queue_size()):
    #     if len(nextGroup_endTime_2) > 0:
    #         next_finish_time = min(nextGroup_endTime_2.values())
    #         next_finish_table = min(nextGroup_endTime_2, key=nextGroup_endTime_2.get)
    #         unpro_next = queue_2.del_queue()
    #         print('Group', unpro_next.get_groupID(), 'needs to wait', 
    #               int(unpro_next.wait_time(next_finish_time)),'minutes to be assigned.')
    #         at_least_waittime.append(int(unpro_next.wait_time(next_finish_time)))
    #         nextGroup_endTime_2.pop(next_finish_table)
    #     else:
    #         unpro_next = queue_2.del_queue()
    #         print('There are still', i, 'Groups in front of Group No.',
    #                unpro_next.get_groupID(), 'you need to wait at least',max(at_least_waittime),'minutes to be assign')
    # #avg_timeR = sum(total_timeR)/len(total_timeR)
        

In [15]:
def stimulation(current_time, table_type, queue_type, total_time, total_timeR, nextGroup_endTime):
    
    if current_time in nextGroup_endTime.values():
        for n in list(nextGroup_endTime.keys()):
            if current_time == int(nextGroup_endTime[n]):
                print('Clean:', n,'\n')
                table_type[n-1].currentGroup = None
                #table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
                #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
                    
        
    for t in table_type:
        if (not t.busy()) and (not queue_type.isEmpty()):
            print('table ID', t.get_num())
            nextGroup = queue_type.del_queue()
            print('Regislation time:', nextGroup.get_stamp())
            print('Wait time:', nextGroup.wait_time(current_time))
            t.startNext(nextGroup)
            print('Group can sit now:', nextGroup.get_groupID())
            nextGroup_endTime[t.get_num()] = current_time+nextGroup.get_time_request()+2  #end time for each 2-people-table
            #2min for table cleaning, rearragement and the time to walk in blabla
            #print(nextGroup_endTime_2[t.get_num()], 'next group end time')
            total_timeR.append(int(nextGroup.get_time_request())+2)  #def generation(Duration):
                                  #print(total_timeR,'total request time \n')
    
    if current_time == total_time-1:
        at_least_waittime = []
        for i in range(queue_type.queue_size()):
            if len(nextGroup_endTime) > 0:
                next_finish_time = min(nextGroup_endTime.values())
                next_finish_table = min(nextGroup_endTime, key=nextGroup_endTime.get)
                unpro_next = queue_type.del_queue()
                print('Group', unpro_next.get_groupID(), 'needs to wait', 
                      int(unpro_next.wait_time(next_finish_time)),'minutes to be assigned.')
                at_least_waittime.append(int(unpro_next.wait_time(next_finish_time)))
                nextGroup_endTime.pop(next_finish_table)
            else:
                unpro_next = queue_type.del_queue()
                print('There are still', i, 'Groups in front of Group No.',
                       unpro_next.get_groupID(), 'you need to wait at least',max(at_least_waittime),'minutes to be assign')
        #avg_timeR = sum(total_timeR)/len(total_timeR)


In [18]:
generation(240) #90 groups

New Regislation vip False
No. 1
Time 34
[1] current queue_2
table ID 0
Regislation time: 34
Wait time: 0
Group can sit now: 1
New Regislation vip False
No. 2
Time 38
[2] current queue_2
table ID 1
Regislation time: 38
Wait time: 0
Group can sit now: 2
New Regislation vip False
No. 3
Time 40
[3] current queue_2
table ID 2
Regislation time: 40
Wait time: 0
Group can sit now: 3
New Regislation vip False
No. 4
Time 55
[4] current queue_2
table ID 3
Regislation time: 55
Wait time: 0
Group can sit now: 4
New Regislation vip False
No. 5
Time 57
[5] current queue_2
table ID 4
Regislation time: 57
Wait time: 0
Group can sit now: 5
New Regislation vip False
No. 6
Time 70
[6] current queue_2
table ID 5
Regislation time: 70
Wait time: 0
Group can sit now: 6
New Regislation vip True
No. 7
Time 72
[7] current queue_2
Clean: 3 

table ID 2
Regislation time: 72
Wait time: 1
Group can sit now: 7
New Regislation vip False
No. 8
Time 79
[8] current queue_2
New Regislation vip False
No. 9
Time 86
[9, 8] c